In [1]:
import cv2
import os,socket,sys,time
import spidev as SPI
import xgoscreen.LCD_2inch as LCD_2inch
from PIL import Image,ImageDraw,ImageFont
from key import Button
import math
import numpy as np
from xgolib import XGO
dog = XGO(port='/dev/ttyAMA0',version="xgolite")

In [2]:
mydisplay = LCD_2inch.LCD_2inch()
mydisplay.clear()
splash = Image.new("RGB", (mydisplay.height, mydisplay.width ),"black")
mydisplay.ShowImage(splash)
button=Button()

In [3]:
#-----------------------COMMON INIT-----------------------
import cv2
import mediapipe as mp
font = cv2.FONT_HERSHEY_SIMPLEX
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

def rotation_matrix_to_angles(rotation_matrix):
    """
    Calculate Euler angles from rotation matrix.
    :param rotation_matrix: A 3*3 matrix with the following structure
    [Cosz*Cosy  Cosz*Siny*Sinx - Sinz*Cosx  Cosz*Siny*Cosx + Sinz*Sinx]
    [Sinz*Cosy  Sinz*Siny*Sinx + Sinz*Cosx  Sinz*Siny*Cosx - Cosz*Sinx]
    [  -Siny             CosySinx                   Cosy*Cosx         ]
    :return: Angles in degrees for each axis
    """
    x = math.atan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
    y = math.atan2(-rotation_matrix[2, 0], math.sqrt(rotation_matrix[0, 0] ** 2 +
                                                     rotation_matrix[1, 0] ** 2))
    z = math.atan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    return np.array([x, y, z]) * 180. / math.pi


In [4]:
# 导入组件 Importing Components
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=320, height=240)  #设置摄像头显示组件  Set up the camera display component

# 将BGR图像转换为JPEG格式的字节流 Convert a BGR image to a JPEG byte stream
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

display(image_widget) #显示出来

Image(value=b'', format='jpeg', height='240', width='320')

In [5]:
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap=cv2.VideoCapture(0)
cap.set(3,320)
cap.set(4,240)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    face_coordination_in_real_world = np.array([
        [285, 528, 200],
        [285, 371, 152],
        [197, 574, 128],
        [173, 425, 108],
        [360, 574, 128],
        [391, 425, 108]
    ], dtype=np.float64)

    h=240
    w=320 
    face_coordination_in_image = []
    text=''
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    direction=0
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
            
        for idx, lm in enumerate(face_landmarks.landmark):
          if idx in [1, 9, 57, 130, 287, 359]:
            x, y = int(lm.x * w), int(lm.y * h)
            face_coordination_in_image.append([x, y])
        face_coordination_in_image = np.array(face_coordination_in_image,dtype=np.float64)
        # The camera matrix
        focal_length = 1 * w
        cam_matrix = np.array([[focal_length, 0, w / 2],
                               [0, focal_length, h / 2],
                               [0, 0, 1]])
        # The Distance Matrix
        dist_matrix = np.zeros((4, 1), dtype=np.float64)
        success, rotation_vec, transition_vec = cv2.solvePnP(face_coordination_in_real_world, face_coordination_in_image,cam_matrix, dist_matrix)
        # Use Rodrigues function to convert rotation vector to matrix
        rotation_matrix, jacobian = cv2.Rodrigues(rotation_vec)
        result = rotation_matrix_to_angles(rotation_matrix)
        print(result)
        pitch=round(-result[0]/100*20)
        yaw=round(result[1]/80*15)
        roll=round(result[2]/80*15)
        if abs(yaw)<= 4:
            if abs(pitch)<3:
                pitch = round( pitch * 7 )
                print("pitch11",pitch)
                if pitch < -7:
                    print("hello")
                    pitch = -3
                    print("pitch",pitch)
                elif pitch > 7:
                    pitch = 3
                    print("picth",pitch)
            else:
                pitch = pitch
                print("pitch",pitch)
                yaw = 0
                roll = 0
        else:
            pitch = -3
            yaw = yaw 
            if abs(roll) >29:
                roll = round(roll/6)
            else:
                roll = roll
        print("pitch,yaw,roll",pitch,yaw,roll)
        dog.attitude(['p','y','r'],[pitch,yaw,roll])
        time.sleep(0.1)
    else:
      pass
    # Flip the image horizontally for a selfie-view mydisplay.
    b,g,r = cv2.split(image)
    image = cv2.merge((r,g,b))
    image = cv2.flip(image, 1)
    try:
      for i, info in enumerate(zip(('Pitch', 'Roll', 'Yaw'), result)):
        k, v = info
        text = f'{k}: {int(v)}'
        cv2.putText(image, text, (20, i*30 + 20),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 0, 200), 2)
    except:
      pass
    imgok = Image.fromarray(image)
    mydisplay.ShowImage(imgok)

    #把结果显示到屏幕上
    r,g,b = cv2.split(image)
    image1 = cv2.merge((b,g,r))
    image_widget.value = bgr8_to_jpeg(image1)
    # cv2.imshow("image",image1)

    if cv2.waitKey(5) & 0xFF == 27:
      break
    if button.press_b():
      break



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[16.34344504 20.14138752 22.19556265]
pitch -3
pitch,yaw,roll -3 0 0
[26.46344102 22.96405539 29.1095638 ]
pitch -5
pitch,yaw,roll -5 0 0
[26.67934674 26.40350639 19.74445784]
pitch,yaw,roll -3 5 4
[26.10123821 21.770443   19.86141382]
pitch -5
pitch,yaw,roll -5 0 0
[26.86498011 29.09827802 18.87568803]
pitch,yaw,roll -3 5 4
[28.45669023 28.64878634 17.76690659]
pitch,yaw,roll -3 5 3
[22.03013824 31.43078269 14.75323772]
pitch,yaw,roll -3 6 3
[24.41734396 30.94456468 16.53883366]
pitch,yaw,roll -3 6 3
[27.20082037 33.29494402 17.27111853]
pitch,yaw,roll -3 6 3
[27.18094749 33.03725687 18.02674422]
pitch,yaw,roll -3 6 3
[26.05481422 39.74350087 16.82983436]
pitch,yaw,roll -3 7 3
[26.43114213 38.40675998 16.59431795]
pitch,yaw,roll -3 7 3
[24.7028922  46.34699163 16.44382979]
pitch,yaw,roll -3 9 3
[24.4989715  45.67915343 14.10301766]
pitch,yaw,roll -3 9 3
[26.56672782 43.42471884 14.25470838]
pitch,yaw,roll -3 8 3
[33.65492386 37.38187011 17.38347227]
pitch,yaw,roll -3 7 3
[29.2543302  

KeyboardInterrupt: 

In [6]:
cap.release()
dog.reset()
del dog